### Import libraries and AWS configs

In [1]:
import os
import boto3
from sagemaker import get_execution_role, Session, s3
from sagemaker.deserializers import JSONDeserializer
from sagemaker.pytorch import PyTorchModel
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig
from time import gmtime, strftime

In [2]:
# Load config file 
from basic_utils.utils import load_config
config = load_config("sagemaker_files/config.yaml")

In [3]:
role = get_execution_role()
sess = Session()
region = sess.boto_region_name
sm_client = boto3.client("sagemaker", region_name=region)

### Create model

In [7]:
# Create model
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

# account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

role = get_execution_role()

In [ ]:

from time import gmtime, strftime

model_name = config.MODEL_NAME 

container = config.IMAGE_URL
instance_type = config.INSTANCE_TYPE

print('Model name: ' + model_name)
# print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

### Clean up (delete endpoint)

In [8]:
# Delete endpoint when you don't need your model deployed
sm_client.delete_endpoint(EndpointName=ENDPOINT_NAME)
sm_client.delete_endpoint_config(EndpointConfigName=ENDPOINT_NAME)

{'ResponseMetadata': {'RequestId': 'f24ec47f-f963-4b51-9c4b-c8ec58ec3eba',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f24ec47f-f963-4b51-9c4b-c8ec58ec3eba',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 17 Apr 2023 06:45:57 GMT'},
  'RetryAttempts': 0}}

Endpoint configuration

In [10]:
# Create Endpoint configuration
print('Endpoint config name: ' + config.ENDPOINT_NAME)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = config.ENDPOINT_NAME,
    ProductionVariants=[{
        'ModelName': model_name,
        "ServerlessConfig": {
                    "MemorySizeInMB": config.MEMORY_SIZE,
                    "MaxConcurrency": config.MAX_CONCURRENCY,
                },
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Model Object Watermark-Logo-Detection-2023-04-17-06-45-53 Created!


### Deploy model to endpoint

In [11]:
%%time

import time

endpoint_name = config.ENDPOINT_NAME
endpoint_config_name = config.ENDPOINT_NAME
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

-----!
Model Watermark-Logo-Detection-2023-04-17-06-45-53 deployed at endpoint watermark-logo-detection!


### Test deployed endpoint

In [12]:
# Inference
import boto3
import cv2

# test using image url
ENDPOINT_NAME = config.ENDPOINT_NAME
payload = "https://images.livspace-cdn.com/plain/https://d3gq2merok8n5r.cloudfront.net/abhinav/ond-1634120396-Obfdc/jfm-2023-1672723560-eFGVH/balcony-1674210268-39YjC/bll-1675062554-QvyMI.jpg"
runtime = boto3.client(service_name='sagemaker-runtime')

import json
content_type = "application/json"
request_body = {'image_url':payload}

#Serialize data for endpoint
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

#Endpoint invocation
response = runtime.invoke_endpoint(
    EndpointName=ENDPOINT_NAME,
    ContentType=content_type,
    Body=payload)

#Parse results
result = json.loads(response['Body'].read().decode())
result

{"detections": "No Logo Watermark Detected"}
{"detections": "No Logo Watermark Detected"}
CPU times: user 110 ms, sys: 21.1 ms, total: 131 ms
Wall time: 7.01 s
